In [1]:
import time
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
driver = webdriver.Chrome()
driver.maximize_window()

In [3]:
final_df = pd.DataFrame()

for i in range(1,44):
    
    driver.get("https://www.flipkart.com/search?q=smartphones&page={}".format(i))

    webpage = driver.page_source

    soup = BeautifulSoup(webpage, 'lxml')

    all_divs = soup.find_all('div', class_="tUxRFH")
    
    
    phone_image_url = []
    phone_name = []
    phone_rating = []
    total_rating = []
    total_review = []
    discount_price = []
    actual_price = []

    for phone in all_divs:
        
        # Image URL
        try:
            phone_image_url.append(phone.find('img', class_='DByuf4')['src'])
        except (AttributeError, KeyError, TypeError):
            phone_image_url.append(np.nan)

        # Phone Name
        try:
            phone_name.append(phone.find('div', class_='KzDlHZ').text.strip())
        except AttributeError:
            phone_name.append(np.nan)

        # Rating
        try:
            phone_rating.append(phone.find('div', class_='XQDdHH').text.strip())
        except AttributeError:
            phone_rating.append(np.nan)

        # Total Rating
        try:
            rating_spans = phone.find_all('span', class_='Wphh3N')[0].find_all('span')[0].find_all('span')
            total_rating.append(rating_spans[0].text.strip())
        except (AttributeError, IndexError):
            total_rating.append(np.nan)

        # Total Review
        try:
            review_spans = phone.find_all('span', class_='Wphh3N')[0].find_all('span')[0].find_all('span')
            total_review.append(review_spans[2].text.strip())
        except (AttributeError, IndexError):
            total_review.append(np.nan)

        # Discount Price
        try:
            discount_price.append(phone.find('div', class_='Nx9bqj _4b5DiR').text.strip())
        except AttributeError:
            discount_price.append(np.nan)

        # Actual Price
        try:
            actual_price.append(phone.find('div', class_='yRaY8j ZYYwLA').text.strip())
        except AttributeError:
            actual_price.append(np.nan)

    df = pd.DataFrame({
        'phone_name': phone_name,
        'discount_price': discount_price,
        'actual_price': actual_price,
        'phone_rating': phone_rating,
        'phone_image_url': phone_image_url,
        'total_rating': total_rating,
        'total_review': total_review,
    })
    
    final_df = pd.concat([final_df, df], ignore_index=True)
    
    phone_image_url.clear()
    phone_name.clear()
    phone_rating.clear()
    total_rating.clear()
    total_review.clear()
    discount_price.clear()
    actual_price.clear()
    
    time.sleep(4)

In [5]:
final_df

,phone_name,discount_price,actual_price,phone_rating,phone_image_url,total_rating,total_review
0,"Infinix Note 50s 5G+ (Marine Drift Blue, 128 GB)","₹15,999","₹21,999",4.5,https://rukminim2.flixcart.com/image/312/312/x...,"1,255 Ratings",168 Reviews
1,"SAMSUNG Galaxy A35 5G (Awesome Iceblue, 256 GB)","₹22,999","₹36,999",4.3,https://rukminim2.flixcart.com/image/312/312/x...,"5,752 Ratings",418 Reviews
2,"SAMSUNG Galaxy F05 (Twilight Blue, 64 GB)","₹6,499","₹9,999",4.2,https://rukminim2.flixcart.com/image/312/312/x...,"46,010 Ratings","2,706 Reviews"
3,"MOTOROLA g05 (Plum Red, 64 GB)","₹6,999","₹9,999",4.3,https://rukminim2.flixcart.com/image/312/312/x...,"21,832 Ratings","1,762 Reviews"
4,"SAMSUNG Galaxy F06 5G (Lit Violet, 64 GB)","₹7,999","₹12,499",4.2,https://rukminim2.flixcart.com/image/312/312/x...,"4,255 Ratings",326 Reviews
...,...,...,...,...,...,...,...
979,"Apple iPhone 15 (Green, 128 GB)","₹63,999","₹69,900",4.6,https://rukminim2.flixcart.com/image/312/312/x...,"2,54,936 Ratings","9,237 Reviews"
980,"SAMSUNG Galaxy F16 5G (Vibing Blue, 128 GB)","₹12,999","₹17,499",4.2,https://rukminim2.flixcart.com/image/312/312/x...,"1,069 Ratings",89 Reviews
981,"SAMSUNG Galaxy A35 5G (Awesome Navy, 256 GB)","₹22,999","₹36,999",4.3,https://rukminim2.flixcart.com/image/312/312/x...,"5,752 Ratings",418 Reviews
982,"SAMSUNG Galaxy S23 FE (Purple, 128 GB)","₹34,999","₹79,999",4.4,https://rukminim2.flixcart.com/image/312/312/x...,"70,794 Ratings","4,364 Reviews"


In [4]:
final_df.to_csv('flipcart_phones_dataset', index=False)